In [3]:
import os
import sys
import logging
import pandas as pd
import datetime


from pandas_profiling import ProfileReport


project_dir = os.path.dirname(os.path.abspath('.')) # get the path of the project, if doesn't work use os.getcwd() instead and use rfind to find the project dir
sys.path.append(os.path.join(project_dir, 'src', 'data'))

from data_manipulator import get_input_data

bucket = 'eseit_bigdata'
directory = 'bigdata'

In [4]:
file = 'gs://{}/{}/data/metadatos-llamadas-urg-y-emer.csv'.format(bucket, directory)
df_metadata = pd.read_csv(file, encoding='latin1', sep=';')
df_metadata

,NOMBRE,DESCRIPCION
0,FECHA_INCIDENTE,Es la fecha el cual se registra la llamada del...
1,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,Es la fecha el cual se inicia el desplazamient...
2,CODIGO LOCALIDAD,Es el código de las 20 localidades de la ciuda...
3,LOCALIDAD,Es la localidad donde sucede el incidente.
4,EDAD,La edad del paciente.
5,UNIDAD,"La descripción de la edad si es en horas, días..."
6,GENERO,Es la distinción de genero del paciente
7,RED,Es la localización a nivel bogota de la red de...
8,TIPO_INCIDENTE,Es la descripción inicial que tipifica el cent...
9,PRIORIDAD,Es la tipificación según la prioridad del inci...


In [5]:
raw_data = get_input_data(filename = 'datos-abiertos-julio-2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14560 entries, 0 to 14559
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   FECHA_INCIDENTE                    14560 non-null  object
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  14560 non-null  object
 2   CODIGO DE LOCALIDAD                14560 non-null  int64 
 3   LOCALIDAD                          14560 non-null  object
 4   EDAD                               14560 non-null  object
 5   UNIDAD                             14560 non-null  object
 6   GENERO                             14560 non-null  object
 7   RED                                14560 non-null  object
 8   TIPO_INCIDENTE                     14560 non-null  object
 9   PRIORIDAD                          14560 non-null  object
 10  MES                                14560 non-null  object
 11  CLASIFICACION FINAL                13035 non-null  object
dtypes: i

In [6]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [7]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy               1593
 Suba                  1548
Engativa               1521
 Bosa                  1122
 Ciudad Bolivar         994
Usaquen                 786
San Cristobal           776
 Rafael Uribe Uribe     741
Fontibon                723
 Puente Aranda          690
Usme                    636
Chapinero               599
Barrios Unidos          516
Los Martires            516
Santa Fe                489
Teusaquillo             464
Tunjuelito              462
 Antonio Nariño         292
 La Candelaria           91
 Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [8]:
# Let's use the function strip(), this property works for string objects in python that removes whitespaces
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               1593
Suba                  1548
Engativa              1521
Bosa                  1122
Ciudad Bolivar         994
Usaquen                786
San Cristobal          776
Rafael Uribe Uribe     741
Fontibon               723
Puente Aranda          690
Usme                   636
Chapinero              599
Barrios Unidos         516
Los Martires           516
Santa Fe               489
Teusaquillo            464
Tunjuelito             462
Antonio Nariño         292
La Candelaria           91
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [9]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [10]:
raw_data.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,NaN
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [11]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)   # here we could apply the same technique as before to avoid errors

SIN_DATO     6621
MASCULINO    4190
FEMENINO     3749
Name: GENERO, dtype: int64

In [12]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    4261
Inconsciente/Paro Cardiorrespiratorio        1623
Dificultad Respiratoria                      1398
Enfermo                                      1131
Convulsiones                                 1011
Heridos                                       747
Trastorno mental                              709
Dolor torácico                                611
Lesiones personales                           547
Intento de suicidio                           534
Patología Gineco - obstétrica                 425
Síntomas gastrointestinales                   344
Accidente cerebro vascular                    323
Caída                                         318
Intoxicaciones                                141
Ideas de suicidio                             111
Violencia Sexual                               41
Muerte Natural                                 39
SIN_DATO                                       39
Incendio estructural                           29


In [13]:
col = 'CLASIFICACION_FINAL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Traslado                       6162
Cancelado                      3637
NaN                            1525
Trasladado por Otro Recurso    1501
Falsa Alarma                   1291
No Ubica                        215
Desistimiento                   137
No Amerita Traslado              64
Fallecido                        28
Name: CLASIFICACION_FINAL, dtype: int64

In [14]:
raw_data[col].fillna('SIN_DATO').value_counts()

Traslado                       6162
Cancelado                      3637
SIN_DATO                       1525
Trasladado por Otro Recurso    1501
Falsa Alarma                   1291
No Ubica                        215
Desistimiento                   137
No Amerita Traslado              64
Fallecido                        28
Name: CLASIFICACION_FINAL, dtype: int64

In [15]:
col = 'CLASIFICACION_FINAL'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,FECHA_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,2019-07-01 00:05:08,2019-07-01 00:11:02,8,Kennedy,39,Años,FEMENINO,Norte,Dificultad Respiratoria,ALTA,JULIO,Traslado
1,2019-07-01 00:06:57,2019-07-01 00:12:56,19,Ciudad Bolivar,69,Años,MASCULINO,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA,JULIO,SIN_DATO
2,2019-07-01 00:09:26,2019-07-01 00:10:26,6,Tunjuelito,25,Años,MASCULINO,Sur,Enfermo,MEDIA,JULIO,Traslado
3,2019-07-01 00:14:42,2019-07-01 00:15:47,8,Kennedy,65,Años,MASCULINO,Sur,Lesiones personales,MEDIA,JULIO,Traslado
4,2019-07-01 00:17:13,2019-07-01 00:31:13,10,Engativa,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Enfermo,MEDIA,JULIO,Cancelado


In [16]:
col = 'FECHA_INCIDENTE'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

2019-07-23 11:59:03    12
2019-07-15 06:26:54    11
2019-07-04 13:32:06    11
2019-07-21 16:26:01    10
2019-07-26 06:31:41    10
                       ..
2019-07-12 11:41:58     1
2019-07-12 11:43:15     1
2019-07-12 11:43:25     1
2019-07-12 11:52:46     1
2019-07-31 23:47:00     1
Name: FECHA_INCIDENTE, Length: 12611, dtype: int64

In [17]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_julio_2019.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 14560
Number of rows after cleaning: 13842


In [19]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_julio_2019.csv', encoding='latin1', sep=';', index=False)

**AGOSTO 2019

In [20]:
raw_data = get_input_data(filename = 'datos-abiertos-agosto-2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18141 entries, 0 to 18140
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   18141 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  18141 non-null  object
 2   CODIGO DE LOCALIDAD                18141 non-null  int64 
 3   LOCALIDAD                          18141 non-null  object
 4   EDAD                               18141 non-null  object
 5   UNIDAD                             18141 non-null  object
 6   GENERO                             18141 non-null  object
 7   RED                                18141 non-null  object
 8   TIPO DE INCIDIDENT                 18141 non-null  object
 9   PRIORIDAD                          18141 non-null  object
 10  MES                                18141 non-null  object
 11  CLASIFICACION FINAL                18138 non-null  object
dtypes: i

In [21]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD',
        'TIPO DE INCIDIDENT'                : 'TIPO_INCIDENTE',
        'CLASIFICACION FINAL'               : 'CLASIFICACION_FINAL'
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [22]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

Kennedy                2717
 Engativa              1805
 Suba                  1678
 Bosa                  1264
Ciudad Bolivar         1220
Puente Aranda           977
 San Cristobal          906
Usaquen                 894
 Rafael Uribe Uribe     892
Fontibon                878
Chapinero               707
Santa Fe                682
 Usme                   637
 Barrios Unidos         608
 Teusaquillo            599
Tunjuelito              595
Los Martires            593
 Antonio Nariño         369
 La Candelaria          119
 Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [23]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2717
Engativa              1805
Suba                  1678
Bosa                  1264
Ciudad Bolivar        1220
Puente Aranda          977
San Cristobal          906
Usaquen                894
Rafael Uribe Uribe     892
Fontibon               878
Chapinero              707
Santa Fe               682
Usme                   637
Barrios Unidos         608
Teusaquillo            599
Tunjuelito             595
Los Martires           593
Antonio Nariño         369
La Candelaria          119
Sumapaz                  1
Name: LOCALIDAD, dtype: int64

In [24]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [25]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [26]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

SIN_DATO     8617
MASCULINO    5143
FEMENINO     4381
Name: GENERO, dtype: int64

In [27]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    6827
Inconsciente/Paro Cardiorrespiratorio        1875
Dificultad Respiratoria                      1504
Enfermo                                      1233
Convulsiones                                 1135
Heridos                                       847
Lesiones personales                           718
Trastorno mental                              670
Dolor torácico                                628
Intento de suicidio                           555
Síntomas gastrointestinales                   418
Accidente cerebro vascular                    401
Patología Gineco - obstétrica                 353
Caída                                         295
Intoxicaciones                                181
Ideas de suicidio                             102
SIN_DATO                                       58
Incendio estructural                           51
Muerte Natural                                 35
Violencia Sexual                               33


In [28]:
col = 'CLASIFICACION_FINAL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Traslado                       7311
Cancelado                      4935
Trasladado por Otro Recurso    1813
Falsa Alarma                   1738
Desistimiento                  1449
No Amerita Traslado             452
No Ubica                        235
Fallecido                       205
NaN                               3
Name: CLASIFICACION_FINAL, dtype: int64

In [29]:
raw_data[col].fillna('SIN_DATO').value_counts()

Traslado                       7311
Cancelado                      4935
Trasladado por Otro Recurso    1813
Falsa Alarma                   1738
Desistimiento                  1449
No Amerita Traslado             452
No Ubica                        235
Fallecido                       205
SIN_DATO                          3
Name: CLASIFICACION_FINAL, dtype: int64

In [30]:
col = 'CLASIFICACION_FINAL'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD,MES,CLASIFICACION_FINAL
0,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
1,809224194,1/08/2019 0:28,1,Usaquen,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Inconsciente/Paro Cardiorrespiratorio,ALTA,AGOSTO,Cancelado
2,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
3,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado
4,809240194,1/08/2019 0:15,11,Suba,SIN_DATO,SIN_DATO,SIN_DATO,Norte,Accidente cerebro vascular,BAJA,AGOSTO,Cancelado


In [31]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_agosto_2019.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 18141
Number of rows after cleaning: 16807


In [33]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_agosto_2019.csv', encoding='latin1', sep=';', index=False)

**SEPTIEMBRE 2019

In [34]:
raw_data = get_input_data(filename = 'datos-abiertos-septiembre-2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17018 entries, 0 to 17017
Data columns (total 10 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   NUMERO_INCIDENTE                   17018 non-null  int64 
 1   FECHA_INICIO_DESPLAZAMIENTO-MOVIL  17018 non-null  object
 2   CODIGO DE LOCALIDAD                17018 non-null  int64 
 3   LOCALIDAD                          17018 non-null  object
 4   EDAD                               17018 non-null  int64 
 5   UNIDAD                             9371 non-null   object
 6   GENERO                             17018 non-null  object
 7   RED                                17018 non-null  object
 8   TIPO_INCIDENTE                     17018 non-null  object
 9   PRIORIDAD                          17018 non-null  object
dtypes: int64(3), object(7)
memory usage: 1.3+ MB
None


In [35]:
# Don't let white spaces between headers to avoid errors during the pipeline
# This is sometimes called a recipe for etl
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD'
    },
    inplace=True
)
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,NaN,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,NaN,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [36]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy               2398
 Suba                  1634
Engativa               1631
 Bosa                  1294
 Ciudad Bolivar        1138
 Puente Aranda          938
 Rafael Uribe Uribe     854
 Usaquen                829
 San Cristobal          826
Fontibon                809
 Chapinero              749
 Barrios Unidos         616
Tunjuelito              603
Los Martires            593
Usme                    572
 Santa Fe               549
 Teusaquillo            540
 Antonio Nariño         362
 La Candelaria           83
Name: LOCALIDAD, dtype: int64

In [37]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2398
Suba                  1634
Engativa              1631
Bosa                  1294
Ciudad Bolivar        1138
Puente Aranda          938
Rafael Uribe Uribe     854
Usaquen                829
San Cristobal          826
Fontibon               809
Chapinero              749
Barrios Unidos         616
Tunjuelito             603
Los Martires           593
Usme                   572
Santa Fe               549
Teusaquillo            540
Antonio Nariño         362
La Candelaria           83
Name: LOCALIDAD, dtype: int64

In [38]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [39]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,NaN,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,NaN,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [40]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

SIN_DATO     7632
MASCULINO    5082
FEMENINO     4304
Name: GENERO, dtype: int64

In [41]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS    6333
INCONSCIENTE/PARO CARDIORRESPIRATORIO        1720
DIFICULTAD RESPIRATORIA                      1386
ENFERMO                                      1137
CONVULSIONES                                 1039
HERIDOS                                       834
INTENTO DE SUICIDIO                           653
DOLOR TORÁCICO                                629
TRASTORNO MENTAL                              621
LESIONES PERSONALES                           593
PATOLOGÍA GINECO - OBSTÉTRICA                 421
SÍNTOMAS GASTROINTESTINALES                   360
ACCIDENTE CEREBRO VASCULAR                    352
CAÍDA                                         334
INTOXICACIONES                                156
IDEAS DE SUICIDIO                             113
VIOLENCIA SEXUAL                               48
INCENDIO ESTRUCTURAL                           40
SIN_DATO                                       32
SANGRADO VAGINAL                               25


In [42]:
col = 'UNIDAD'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Años     9311
NaN      7647
Meses      42
Dias       10
Horas       8
Name: UNIDAD, dtype: int64

In [43]:
raw_data[col].fillna('SIN_DATO').value_counts()

Años        9311
SIN_DATO    7647
Meses         42
Dias          10
Horas          8
Name: UNIDAD, dtype: int64

In [44]:
col = 'UNIDAD'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

,NUMERO_INCIDENTE,FECHA_INICIO_DESPLAZAMIENTO_MOVIL,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,538006196,2019-09-01 00:05:32,18,Rafael Uribe Uribe,53,Años,MASCULINO,Sur,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
1,538024196,2019-09-01 00:12:07,11,Suba,0,SIN_DATO,SIN_DATO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
2,538031196,2019-09-01 00:05:39,8,Kennedy,26,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
3,538084196,2019-09-01 00:13:39,11,Suba,24,Años,MASCULINO,Norte,ACCIDENTE DE TRÁNSITO CON HERIDOS/MUERTOS,ALTA
4,538123196,2019-09-01 00:20:31,2,Chapinero,0,SIN_DATO,SIN_DATO,Norte,INCONSCIENTE/PARO CARDIORRESPIRATORIO,ALTA


In [45]:
col = 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

2019-09-18 05:09:15    9
2019-09-06 14:47:18    8
2019-09-07 18:29:25    7
2019-09-26 15:06:05    7
2019-09-06 07:53:53    6
                      ..
2019-09-11 10:10:04    1
2019-09-11 10:19:54    1
2019-09-11 10:34:31    1
2019-09-11 10:24:38    1
2019-09-30 23:50:06    1
Name: FECHA_INICIO_DESPLAZAMIENTO_MOVIL, Length: 15925, dtype: int64

In [46]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_septiembre_2019.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

Number of rows before cleaning: 17018
Number of rows after cleaning: 16267


In [48]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_septiembre_2019.csv', encoding='latin1', sep=';', index=False)

**OCTUBRE 2019

In [49]:
raw_data = get_input_data(filename = 'datos-abiertos-octubre-2019.csv')
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18195 entries, 0 to 18194
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NUMERO_INCIDENTE     18195 non-null  int64 
 1   FECHA_DESPACHO_518   18195 non-null  object
 2   CODIGO DE LOCALIDAD  18195 non-null  int64 
 3   LOCALIDAD            18195 non-null  object
 4   EDAD                 18195 non-null  object
 5   UNIDAD               9503 non-null   object
 6   GENERO               9543 non-null   object
 7   RED                  18195 non-null  object
 8   TIPO_INCIDENTE       18195 non-null  object
 9   PRIORIDAD            18195 non-null  object
dtypes: int64(2), object(8)
memory usage: 1.4+ MB
None


In [50]:
raw_data.rename(
    columns = {
        'FECHA_DESPACHO_518'                : 'FECHA_DESPACHO',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD'
    },
    inplace=True
)
raw_data.head(10)

,NUMERO_INCIDENTE,FECHA_DESPACHO,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA
5,868486196,2019-10-01 00:32:35,8,Kennedy,SIN_DATO,NaN,NaN,Norte,Lesiones personales,ALTA
6,868496196,2019-10-01 00:44:52,8,Kennedy,22,Años,MASCULINO,Norte,Convulsiones,ALTA
7,868557196,2019-10-01 00:53:04,8,Kennedy,33,Años,MASCULINO,Norte,Intento de suicidio,BAJA
8,944403194,2019-10-01 05:45:44,16,Puente Aranda,46,Años,FEMENINO,Privada,Accidente de tránsito con heridos/Muertos,ALTA
9,954033194,2019-10-01 23:04:50,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Caída,ALTA


In [51]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

 Kennedy               2546
 Engativa              1791
 Suba                  1742
 Bosa                  1334
 Ciudad Bolivar        1141
 Puente Aranda          989
 Rafael Uribe Uribe     949
Usaquen                 888
 Fontibon               863
San Cristobal           844
 Chapinero              763
 Los Martires           697
 Santa Fe               689
 Barrios Unidos         680
 Usme                   609
 Tunjuelito             589
 Teusaquillo            536
 Antonio Nariño         419
 La Candelaria          120
 Sumapaz                  6
Name: LOCALIDAD, dtype: int64

In [52]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

Kennedy               2546
Engativa              1791
Suba                  1742
Bosa                  1334
Ciudad Bolivar        1141
Puente Aranda          989
Rafael Uribe Uribe     949
Usaquen                888
Fontibon               863
San Cristobal          844
Chapinero              763
Los Martires           697
Santa Fe               689
Barrios Unidos         680
Usme                   609
Tunjuelito             589
Teusaquillo            536
Antonio Nariño         419
La Candelaria          120
Sumapaz                  6
Name: LOCALIDAD, dtype: int64

In [53]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [54]:
raw_data.head()

,NUMERO_INCIDENTE,FECHA_DESPACHO,CODIGO_LOCALIDAD,LOCALIDAD,EDAD,UNIDAD,GENERO,RED,TIPO_INCIDENTE,PRIORIDAD
0,868419196,2019-10-01 00:06:29,9,Fontibon,SIN_DATO,NaN,NaN,Norte,Dolor torácico,ALTA
1,868421196,2019-10-01 00:19:40,10,Engativa,67,Años,MASCULINO,Norte,Heridos,ALTA
2,868438196,2019-10-01 00:13:20,4,San Cristobal,SIN_DATO,NaN,NaN,Sur,Inconsciente/Paro Cardiorrespiratorio,ALTA
3,944374194,2019-10-01 05:30:23,16,Puente Aranda,SIN_DATO,NaN,NaN,Privada,Convulsiones,ALTA
4,868476196,2019-10-01 00:42:45,5,Usme,5,Años,FEMENINO,Sur,Dificultad Respiratoria,ALTA


In [55]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

Accidente de tránsito con heridos/Muertos    6778
Inconsciente/Paro Cardiorrespiratorio        1768
Dificultad Respiratoria                      1497
Enfermo                                      1276
Convulsiones                                 1130
Heridos                                       828
Dolor torácico                                735
Intento de suicidio                           668
Lesiones personales                           657
Trastorno mental                              654
Accidente cerebro vascular                    396
Patología Gineco - obstétrica                 378
Síntomas gastrointestinales                   375
Caída                                         322
Intoxicaciones                                179
Ideas de suicidio                             111
SIN_DATO                                       60
Incendio estructural                           51
Muerte Natural                                 39
Violencia Sexual                               32


In [56]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

AttributeError: 'float' object has no attribute 'strip'

In [ ]:
col = 'UNIDAD'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
raw_data[col].fillna('SIN_DATO').value_counts()

In [ ]:
col = 'UNIDAD'
raw_data[col].fillna('SIN_DATO', inplace=True)
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data.head()

In [ ]:
col = 'FECHA_DESPACHO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_ocutbre_2019.html')

In [ ]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

In [ ]:
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_octubre_2019.csv', encoding='latin1', sep=';', index=False)

**NOVIEMBRE 2019

In [ ]:
raw_data = get_input_data(filename = 'datos-abiertos-noviembre_2019.csv')
print(raw_data.info())

In [ ]:
raw_data.rename(
    columns = {
        'FECHA_INICIO_DESPLAZAMIENTO-MOVIL' : 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD'
    },
    inplace=True
)
raw_data.head()

In [ ]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

In [ ]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

In [ ]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip()) # we assign the output of the function apply to the same column

In [ ]:
raw_data.head()

In [ ]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
col = 'TIPO_INCIDENTE'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
col = 'UNIDAD'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
col = 'PRIORIDAD'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
col = 'FECHA_INICIO_DESPLAZAMIENTO_MOVIL'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
raw_data = pd.to_datetime('2019-11-10', format = '%yyyy-%mm-%dd')

In [ ]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_noviembre_2019.html')

In [ ]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

In [ ]:
# saving in a local directory
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_noviembre_2019.csv', encoding='latin1', sep=';', index=False)

**DICIEMBRE 2019

In [ ]:
raw_data = get_input_data(filename = 'datos_abiertos_cons_diciembre_2019.csv')
print(raw_data.info())

In [ ]:
raw_data.rename(
    columns = {
        'FECHA_DESPACHO_518'                : 'FECHA_DESPACHO',
        'CODIGO DE LOCALIDAD'               : 'CODIGO_LOCALIDAD'
    },
    inplace=True
)
raw_data.head()

In [ ]:
raw_data['LOCALIDAD'].value_counts(dropna=False)

In [ ]:
raw_data['LOCALIDAD'].apply(
    lambda x: x.strip()        # here we use a lambda function, exclusively for python. Is a one-line function
).value_counts()

In [ ]:
raw_data['LOCALIDAD'] = raw_data['LOCALIDAD'].apply(lambda x: x.strip())

In [ ]:
raw_data.head()

In [ ]:
col = 'GENERO'
raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
col = 'UNIDAD'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
col = 'PRIORIDAD'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
col = 'FECHA_DESPACHO'
# raw_data[col] = raw_data[col].apply(lambda x: x.strip())
raw_data[col].value_counts(dropna=False)

In [ ]:
profile = ProfileReport(df=raw_data)
profile.to_file(output_file=project_dir + '/reports/'+'profile_raw_data_diciembre_2019.html')

In [ ]:
print('Number of rows before cleaning:', raw_data.shape[0] )
raw_data.drop_duplicates(inplace=True)
print('Number of rows after cleaning:', raw_data.shape[0] )

In [ ]:
raw_data.to_csv(project_dir + '/data/processed/' +'clean_llamadas_123_diciembre_2019.csv', encoding='latin1', sep=';', index=False)

In [ ]:
arc1 = pd.read_csv('clean_llamadas_123_agosto_2019.csv', index_col=0)

In [ ]:
dataclean = pd.concat(['clean_llamadas_123_agosto_2019.csv', 'clean_llamadas_123_diciembre_2019.csv', 'clean_llamadas_123_julio_2019.csv', 'clean_llamadas_123_noviembre_2019.csv', 'clean_llamadas_123_octubre_2019.csv', 'clean_llamadas_123_septiembre_2019.csv']) #concat = concatenar, head = el numero de filas que va a tomar
print(dataclean)